In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Popolazione_italiana.csv")
italy_df = df.loc[df['Territorio'] == "Italia"]
age_tot = italy_df.loc[(italy_df['SEXISTAT1'] == 9) & (italy_df['STATCIV2'] == 99)]

In [3]:
df_age = pd.DataFrame(data=age_tot[['Età','Value']])
df_age.insert(2, "Age", [i for i in range(len(df_age))])
df_age.drop("Età", axis=1, inplace=True)
df_age.set_index('Age', inplace=True)
df_age.drop(index=101, inplace=True)
df_age.insert(1, 'Age_category', [abs(i)//10 for i in range(-1,len(df_age)-1)])
df_age.tail(20)
age = df_age.groupby("Age_category").sum()

In [4]:
N = age.values
N = N[:,0]
N.shape

(10,)

In [5]:
df_icu = pd.read_csv("ReportISS_ICU.txt", sep="\t")
df_icu.loc[(df_icu.CASI == '<5'), 'CASI'] = 0
df_cases = pd.to_numeric(df_icu['CASI'])
sum(df_cases.values) 
#TODO: these data seem weird with respect to what we get in the infographic of sole (coming from
#protezione civile. Just notice this we can't do much more unfortunately)
#Notice that we can get the recovery rate by taking the cumulative cases from the beginning of 
#the epidemic

#we can still use this as an estimate for the entering rate of ICU for age category:
# ICU = CRITICO/Total cases

644035

In [6]:
sum(df_cases.values)

644035

In [7]:
df_icu_critico = df_icu.loc[df_icu['STATO_CLINICO'] == 'CRITICO']
df_icu_critico = df_icu_critico[['AGE_GROUP', 'CASI']]
df_icu_critico['CASI'] = pd.to_numeric(df_icu_critico['CASI'])
#df_icu_critico = df_icu_critico.groupby('AGE_GROUP').sum()
df_icu_critico = df_icu_critico.groupby('AGE_GROUP').sum()

In [8]:
df_icu = df_icu[['AGE_GROUP', 'CASI']]
df_icu['CASI'] = pd.to_numeric(df_icu['CASI'])
df_icu = df_icu.groupby('AGE_GROUP').sum()

In [9]:
icu = df_icu_critico.values[:,0]
tot = df_icu.values[:,0]

In [10]:
icu_ratio = np.zeros(len(icu))
for i in range(len(icu)):
    icu_ratio[i] = icu[i]/tot[i]
icu_ratio #These numbers are weird? I would expect a monotonic increase of entrances in ICU. 
#We'll think about this tomorrow anyway

array([2.19106047e-04, 9.02885622e-05, 3.55429181e-04, 7.73134812e-04,
       1.75085792e-03, 4.70404793e-03, 1.27731934e-02, 1.96561979e-02,
       9.40152950e-03, 4.52048126e-03])

In [11]:
df_infected = pd.read_csv("ReportISS_ICU.txt", sep="\t")
df_infected = df_infected[["AGE_GROUP", "CASI"]]
df_infected.loc[(df_infected.CASI == '<5'), 'CASI'] = 0
df_infected['CASI'] = pd.to_numeric(df_infected['CASI'])
df_infected = df_infected.groupby('AGE_GROUP').sum()
infected_age = df_infected.values[:-1]

In [12]:
infected_age

array([[ 27384],
       [ 55378],
       [ 78778],
       [ 80193],
       [ 99951],
       [114157],
       [ 75862],
       [ 55148],
       [ 42759],
       [ 14379]])

In [13]:
df_cases = pd.read_csv("ReportISS_CumulativeCasesDeaths_age.txt", sep="\t")
df_cases.loc[(df_cases.DECEDUTI == '<5'), 'DECEDUTI'] = 0
df_cases = df_cases[df_cases.SESSO != 'Non noto']
df_cases = df_cases[df_cases.AGE_GROUP != 'Non noto']
df_cases['CASI_CUMULATIVI'] = pd.to_numeric(df_cases['CASI_CUMULATIVI'])
df_cases['DECEDUTI'] = pd.to_numeric(df_cases['DECEDUTI'])

df_cum_cases = df_cases[['AGE_GROUP', 'CASI_CUMULATIVI']]
df_cum_cases = df_cum_cases.groupby('AGE_GROUP').sum()

df_deads = df_cases[['AGE_GROUP', 'DECEDUTI']]
df_deads = df_deads.groupby('AGE_GROUP').sum()

cases = df_cum_cases.values
deads = df_deads.values
